# 코드 한번에 실행하면 오류가 발생하므로 하나씩 실행시키기

In [1]:
# 네이버 데이터랩 홈페이지 접속하기 
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import sys
import os
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

driver = webdriver.Chrome('C:/Myexam/chromedriver/chromedriver.exe') 
url = 'https://datalab.naver.com/local/card_result.naver'
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

C:\Users\user\AppData\Local\Temp/ipykernel_14988/1719304812.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Myexam/chromedriver/chromedriver.exe')


In [2]:
# 전체 선택 버튼 누르기(처음엔 선택이 안 되어있음)

choose_all = driver.find_element(By.CSS_SELECTOR,'#item_2_all').click()

In [3]:
# 이후 조회하기 버튼 누르면 서울 전 자치구에 대한 카드사용량 그래프가 나타난다

lookup = driver.find_element(By.CSS_SELECTOR,'#content > div.section_trend > div.com_box_inner > form > fieldset > a').click()

In [4]:
contents = driver.find_elements(By.CSS_SELECTOR,'#content > div.section_graph > div.com_box_inner > div > div.graph_cate.full > .cate_list')
len(contents)

10

In [5]:
contents[0].find_element(By.CSS_SELECTOR,'div > label').text # 잘 나오는 것 확인

'강남구'

In [6]:
districts = []
for content in contents:
    district = content.find_element(By.CSS_SELECTOR,'div > label').text
    districts.append(district)
    
print(districts)

['강남구', '마포구', '서초구', '영등포구', '중구', '종로구', '송파구', '용산구', '강서구', '관악구']


In [7]:
# 상위 3개의 구 슬라이스
# 리스트 순서가 카드 사용이 많은 순이다

dataset = districts[:3]

print(dataset)

['강남구', '마포구', '서초구']


In [8]:
driver.quit() # 상위 3개 자치구 선정 끝

In [9]:
# 크롬 드라이버 

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

C:\Users\user\AppData\Local\Temp/ipykernel_14988/2911649310.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [10]:
dataset_total = []

def extract():
    for i in range(1,11):
        for j in range(1,3):
            # 가게 이름
            place_title = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > a > h2'
            title = driver.find_element(By.CSS_SELECTOR, place_title)
            # 수집 확인
            # print(title.text)


            # 가게 점수
            place_score = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > strong'
            score = driver.find_element(By.CSS_SELECTOR, place_score)
            # 수집 확인
            # print(score.text)


            # 가게 위치 및 카테고리
            place_address_category = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > p.etc'
            address_category = driver.find_element(By.CSS_SELECTOR, place_address_category)
            temp = address_category.text
            temp_list = re.split(' - ', temp)
            address = temp_list[0]
            category = temp_list[1]
            #수집 확인
            # print(address)
            # print(category)


            # 가게 리뷰 수
            place_evaluation = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > p.etc_info > span.review_count'
            evaluation = driver.find_element(By.CSS_SELECTOR, place_evaluation)
            # 수집 확인
            # print(evealuation.text)


            # 가게 조회 수
            place_view = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > p.etc_info > span.view_count'
            view = driver.find_element(By.CSS_SELECTOR, place_view)
            # 수집 확인
            # print(view.text)
            dataset_total.append([title.text,score.text,address,category,evaluation.text,view.text])

In [11]:
# 데이터 셋의 1~10 페이지를 순회하며 추출합니다.
# 현재 각 페이지의 첫번째 가게만 추출합니다.

for i in dataset:
    for j in range(1,11):
        # {0} : 검색어(;지역), {1} : 페이지 입니다.
        driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(i, j))
        extract()

In [12]:
columns = ['이름','평점','위치','카테고리','리뷰 수','조회 수']
pd_data = pd.DataFrame(dataset_total, columns=columns)
pd_data

,이름,평점,위치,카테고리,리뷰 수,조회 수
0,미라이,4.8,가로수길,이자카야 / 오뎅 / 꼬치,122,"456,964"
1,시라카와,4.8,신사/압구정,이자카야 / 오뎅 / 꼬치,67,"279,464"
2,페리지,4.8,삼성동,기타 양식,35,"24,934"
3,맛짱조개,4.8,논현동,해산물 요리,59,"111,305"
4,상진식감,4.8,가로수길,까스 요리,19,"10,163"
...,...,...,...,...,...,...
595,스시코에,4.0,방배/반포/잠원,회 / 스시,14,"187,649"
596,스시소라 (서초점),4.0,교대/서초,정통 일식 / 일반 일식,11,"12,918"
597,카페드앙떼띠 (휴업중),4.0,서래마을,카페 / 디저트,49,"37,976"
598,묘오또,4.0,방배/반포/잠원,라멘 / 소바 / 우동,51,"19,470"


In [13]:
driver.quit()

In [14]:
dataset_gangnam = pd_data[:200]
dataset_mapo = pd_data[200:400]
dataset_seocho = pd_data[400:]

In [15]:
dataset_gangnam

,이름,평점,위치,카테고리,리뷰 수,조회 수
0,미라이,4.8,가로수길,이자카야 / 오뎅 / 꼬치,122,"456,964"
1,시라카와,4.8,신사/압구정,이자카야 / 오뎅 / 꼬치,67,"279,464"
2,페리지,4.8,삼성동,기타 양식,35,"24,934"
3,맛짱조개,4.8,논현동,해산물 요리,59,"111,305"
4,상진식감,4.8,가로수길,까스 요리,19,"10,163"
...,...,...,...,...,...,...
195,양식당더램키친 (2호점),4.3,가로수길,고기 요리,68,"133,420"
196,잭슨피자 (압구정점),4.3,신사/압구정,기타 양식,45,"52,762"
197,몰타참치,4.3,대치동,회 / 스시,44,"110,444"
198,심양양꼬치 (청담점),4.3,신사/압구정,기타 중식,60,"45,538"


In [16]:
dataset_mapo

,이름,평점,위치,카테고리,리뷰 수,조회 수
200,쉐시몽,4.8,홍대,프랑스 음식,44,"121,725"
201,카와카츠 (합정점),4.8,합정/망원,까스 요리,65,"48,010"
202,야키토리나루토,4.8,홍대,정통 일식 / 일반 일식,23,"30,901"
203,카와카츠,4.7,홍대,까스 요리,184,"190,568"
204,코바시,4.7,합정/망원,정통 일식 / 일반 일식,11,"13,987"
...,...,...,...,...,...,...
395,이와나시,4.2,연남동,정통 일식 / 일반 일식,25,"37,469"
396,헬로인디아,4.2,연남동,인도 음식,48,"57,158"
397,단샤리,4.2,연남동,칵테일 / 와인,4,582
398,천지양꼬치,4.2,합정/망원,기타 중식,37,"34,970"


In [17]:
dataset_seocho

,이름,평점,위치,카테고리,리뷰 수,조회 수
400,타쿠미곤,4.7,서래마을,정통 일식 / 일반 일식,62,"239,144"
401,마루심,4.7,방배/반포/잠원,돈부리 / 일본 카레 / 벤토,218,"462,852"
402,메종조,4.7,교대/서초,프랑스 음식,27,"59,195"
403,밴건디 스테이크 하우스,4.6,서래마을,스테이크 / 바베큐,141,"159,650"
404,서관면옥,4.6,교대/서초,국수 / 면 요리,165,"129,882"
...,...,...,...,...,...,...
595,스시코에,4.0,방배/반포/잠원,회 / 스시,14,"187,649"
596,스시소라 (서초점),4.0,교대/서초,정통 일식 / 일반 일식,11,"12,918"
597,카페드앙떼띠 (휴업중),4.0,서래마을,카페 / 디저트,49,"37,976"
598,묘오또,4.0,방배/반포/잠원,라멘 / 소바 / 우동,51,"19,470"


In [18]:
dataset_gangnam.to_csv("C:/Users/user/Desktop/DV_project/csv_files/Gangnam.csv",index=False,encoding='cp949')
dataset_mapo.to_csv("C:/Users/user/Desktop/DV_project/csv_files/Mapo.csv",index=False,encoding='cp949')
dataset_seocho.to_csv("C:/Users/user/Desktop/DV_project/csv_files/Seocho.csv",index=False,encoding='cp949')